In [39]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost.sklearn import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from cslib import fetch_ts, engineer_features

# 1. Loading Data

In [2]:
data_dir = os.path.join("data","cs-train")
ts_all = fetch_ts(data_dir,clean=False)
ts_all['all'].head()

... loading ts data from files


,date,purchases,unique_invoices,unique_streams,total_views,year_month,revenue
0,2017-11-01,0,0,0,0,2017-11,0.0
1,2017-11-02,0,0,0,0,2017-11,0.0
2,2017-11-03,0,0,0,0,2017-11,0.0
3,2017-11-04,0,0,0,0,2017-11,0.0
4,2017-11-05,0,0,0,0,2017-11,0.0


# 2. Feature Engineering and Train Test Split

### Generate features using 7, 14, 28, 70-day time window wraping, the monthly sum of previous year, the average number of invoices and the total views in rencent 30 days

In [3]:
X,y,dates = engineer_features(ts_all['all'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [4]:
print(X.shape, y.shape)

(549, 7)

# 3. Model Training and Perforamcne Comparision  

### 3.1. Gradient Boosting Regressor

In [6]:
param_grid_gb = {
    'gb__criterion': ['mse','mae'],
    'gb__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_gb = Pipeline(steps=[('scaler', StandardScaler()), ('gb', GradientBoostingRegressor())])

grid = GridSearchCV(pipe_gb, param_grid=param_grid_gb, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

gb_mae =  mean_absolute_error(y_test, y_pred)
gb_mse =  mean_squared_error(y_test, y_pred)
gb_r2_score = r2_score(y_test, y_pred)
gb_explained_variance_score = explained_variance_score(y_test, y_pred)

print("mae = {:.2f}".format(gb_mae))
print("mse = {:.2f}".format(gb_mse))
print("r2_score = {:.3f}".format(gb_r2_score))
print("best params =", grid.best_params_)
print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("--------------------------------------------------------------------------------------")

mae = 16228.58
mse = 455006311.93
r2_score = 0.931
explained_variance_score = 0.931
best params = {'gb__criterion': 'mse', 'gb__n_estimators': 100}
train time =  00:00:14
--------------------------------------------------------------------------------------


### 3.2. Random Forest Regressor

In [7]:
param_grid_rf = {
    'rf__criterion': ['mse','mae'],
    'rf__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_rf = Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rf_mae =  mean_absolute_error(y_test, y_pred)
rf_mse =  mean_squared_error(y_test, y_pred)
rf_r2_score = r2_score(y_test, y_pred)
rf_explained_variance_score = explained_variance_score(y_test, y_pred)

print("mae = {:.2f}".format(rf_mae))
print("mse = {:.2f}".format(rf_mse))
print("r2_score = {:.3f}".format(rf_r2_score))
print("best params =", grid.best_params_)
print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("--------------------------------------------------------------------------------------")

mae = 11731.89
mse = 279920599.40
r2_score = 0.957
explained_variance_score = 0.958
best params = {'rf__criterion': 'mse', 'rf__n_estimators': 100}
train time =  00:00:09
--------------------------------------------------------------------------------------


### 3.3. Multilayer Perceptron (MLP) Regressor
A multilayer perceptron (MLP) is also known as a vanilla neural network because it is the core example of an architecture. The vanilla neural networks often only have a single hidden layer, but a MLP can have many more. The number of hidden layers and the size (number of nodes in each) are configurable parameters that you will need to keep in mind when building neural networks.

In [8]:
rs = 5
param_grid = {
    'nn__activation': ['relu'],
    'nn__solver': ['lbfgs', 'sgd'],
    'nn__hidden_layer_sizes': [(10,10), (50,50), (64, 64)]
    }

time_start = time.time()
pipe  = Pipeline(steps=[('scaler', StandardScaler()),
                            ('nn', MLPRegressor(alpha=1e-5, random_state=rs, max_iter=5000))])


grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

nn_mae =  mean_absolute_error(y_test, y_pred)
nn_mse =  mean_squared_error(y_test, y_pred)
nn_r2_score = r2_score(y_test, y_pred)
nn_explained_variance_score = explained_variance_score(y_test, y_pred)

print("mae = {:.2f}".format(nn_mae))
print("mse = {:.2f}".format(nn_mse))
print("r2_score = {:.3f}".format(nn_r2_score))
print("best params =", grid.best_params_)
print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("--------------------------------------------------------------------------------------")


/Users/yee/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: RuntimeWarning: overflow encountered in square
  array_means[:, np.newaxis]) ** 2,


mae = 13951.61
mse = 403412292.48
r2_score = 0.939
explained_variance_score = 0.939
best params = {'nn__activation': 'relu', 'nn__hidden_layer_sizes': (10, 10), 'nn__solver': 'lbfgs'}
train time =  00:01:19
--------------------------------------------------------------------------------------


### 3.4. Extreme Gradient Boosting Regressor

In [93]:
# Create the grid
xgb_n_estimators = [int(x) for x in np.linspace(200, 2000, 5)] # Number of trees to be used
xgb_max_depth = [int(x) for x in np.linspace(2, 20, 5)] # Maximum number of levels in tree
xgb_min_child_weight = [int(x) for x in np.linspace(1, 10, 5)] # Minimum number of instaces needed in each node
xgb_tree_method = ['auto', 'exact', 'approx', 'hist', 'gpu_hist'] # Tree construction algorithm used in XGBoost
xgb_eta = [x for x in np.linspace(0.1, 0.6, 3)] # Learning rate
xgb_gamma = [int(x) for x in np.linspace(0, 0.5, 3)] # Minimum loss reduction required to make further partition
xgb_objective = ['reg:squarederror', 'reg:squaredlogerror'] # Learning objective used
xgb_grid = {'xgb__n_estimators': xgb_n_estimators,
            'xgb__max_depth': xgb_max_depth,
            'xgb__min_child_weight': xgb_min_child_weight,
            'xgb__tree_method': xgb_tree_method,
            'xgb__eta': xgb_eta,
            'xgb__gamma': xgb_gamma,
            'xgb__objective': xgb_objective}

#xgb = XGBRegressor()
# Create the random search Random Forest
pipe  = Pipeline(steps=[('scaler', StandardScaler()),
                        ('xgb', XGBRegressor())])
xgb_random = RandomizedSearchCV(pipe, param_distributions = xgb_grid, 
                                n_iter = 100, cv = 3, verbose = 2, 
                                random_state = 42, n_jobs = -1)

# Fit the random search model
xgb_random.fit(X_train, y_train)
y_pred = xgb_random.predict(X_test)

xgb_mae =  mean_absolute_error(y_test, y_pred)
xgb_mse =  mean_squared_error(y_test, y_pred)
xgb_r2_score = r2_score(y_test, y_pred)
print("mae = {:.2f}".format(xgb_mae))
print("mse = {:.2f}".format(xgb_mse))
print("r2_score = {:.3f}".format(xgb_r2_score))
print("best params =", xgb_random.best_params_)
print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("--------------------------------------------------------------------------------------")


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.0min finished


mae = 10986.26
mse = 291398062.93
r2_score = 0.956
best params = {'xgb__tree_method': 'approx', 'xgb__objective': 'reg:squarederror', 'xgb__n_estimators': 650, 'xgb__min_child_weight': 3, 'xgb__max_depth': 15, 'xgb__gamma': 0, 'xgb__eta': 0.1}
train time =  05:16:51
--------------------------------------------------------------------------------------


In [95]:
def regressor_comparison(models, test_features, test_labels):
    scores = pd.DataFrame()
    for model in models:
        predictions = model.predict(test_features)
        mae = mean_absolute_error(test_labels, predictions)
        mse = mean_squared_error(test_labels, predictions)
        r2 = r2_score(test_labels, predictions)
        errors = abs(predictions - test_labels)
        mape = 100 * np.mean(errors / test_labels)
        accuracy = 100 - mape
        scores[str(model)] = [mae, mse, r2, accuracy]
    scores.index = ['Mean Absolute Error', 'Mean Squared Error', 'R^2', 'Accuracy']
    return scores

In [96]:
linear_pipe = Pipeline(steps=[('scaler', StandardScaler()),
                        ('linear', LinearRegression())])

xgb_pipe =  Pipeline(steps=[('scaler', StandardScaler()),
                        ('xgb', XGBRegressor(tree_method = 'approx',
                         objective = 'reg:squarederror',
                         n_estimators = 650,
                         min_child_weight = 3,
                         max_depth = 15,
                         gamma = 0,
                         eta = 0.1,
                         random_state = 42))])

rf_pipe = Pipeline(steps=[('scaler', StandardScaler()),
                          ('rf', RandomForestRegressor(criterion='mse', n_estimators=100))])

gb_pipe = Pipeline(steps=[('scaler', StandardScaler()),
                          ('gb', GradientBoostingRegressor(criterion='mse', n_estimators=100))])

linear_final=linear_pipe.fit(X_train, y_train)
xgb_final=xgb_pipe.fit(X_train, y_train)
rf_final=rf_pipe.fit(X_train, y_train)
gb_final=gb_pipe.fit(X_train, y_train)

In [97]:
# Call the comparison function with the three final models
reg_scores = regressor_comparison([linear_final, xgb_final, rf_final, gb_final], X_test, y_test)
reg_scores.columns  = ['Linear Regression', 'Extreme Gradient Boosting', 'Random Forest', 'Gradient Boosting']
reg_scores

,Linear Regression,Extreme Gradient Boosting,Random Forest,Gradient Boosting
Mean Absolute Error,4.345218e+04,1.098626e+04,1.135305e+04,1.627313e+04
Mean Squared Error,3.086799e+09,2.913981e+08,2.622274e+08,4.552113e+08
R^2,5.295008e-01,9.555842e-01,9.600305e-01,9.306153e-01
Accuracy,7.407875e+01,9.401217e+01,9.370295e+01,9.054586e+01
